# <div style="padding: 30px; color:white; margin:10; font-size:95%; text-align:left; display:fill; border-radius:10px; background-color:#3b3745"><b> Intel Review - Summarization of Techinical Reviews</b></div>

<div style="background-color: #3b3745; border-radius: 12px; padding: 20px; box-shadow: 0 4px 8px 0 rgba(0,0,0,0.2);">
    <h2 style="color: #F1A424; text-align: center;">Table of Contents</h2>
    <ul style="list-style: none; padding: 0;">
        <li><a href="#section-1" style="color: white; text-decoration: none; display: flex; align-items: center; padding: 8px 15px; border-radius: 6px; transition: background-color 0.3s;"><span style="margin-right: 10px; font-weight: bold; color: #F1A424;">1.</span>  Importing Libraries</a></li>
        <li><a href="#section-2" style="color: white; text-decoration: none; display: flex; align-items: center; padding: 8px 15px; border-radius: 6px; transition: background-color 0.3s;"><span style="margin-right: 10px; font-weight: bold; color: #F1A424;">2</span>Loading the CSV file in dataframe </a></li>
        <li><a href="#section-3" style="color: white; text-decoration: none; display: flex; align-items: center; padding: 8px 15px; border-radius: 6px; transition: background-color 0.3s;"><span style="margin-right: 10px; font-weight: bold; color: #F1A424;">3</span> Summarization of Tech Reviews</a></li>    
    </ul>
</div>

<a id="section-1"></a>
# <div style="padding: 30px; color:white; margin:10; font-size:75%; text-align:left; display:fill; border-radius:10px; background-color:#3b3745"><b><span style='color:#F1A424'>1 |</span></b> <b>  Importing Libraries</b></div>

In [ ]:
import pandas as pd
from transformers import pipeline
import math
from tqdm import tqdm


 <a id="section-2"></a>
# <div style="padding: 30px; color:white; margin:10; font-size:75%; text-align:left; display:fill; border-radius:10px; background-color:#3b3745"><b><span style='color:#F1A424'> 2 |</span></b> <b>Loading the CSV file in dataframe 
</b></div>


In [ ]:
input_file = '2-technical-reviews.csv'  
output_file = '3-summarized-reivew.csv' 

df = pd.read_csv(input_file, encoding='ISO-8859-1')


 <a id="section-3"></a>
# <div style="padding: 30px; color:white; margin:10; font-size:75%; text-align:left; display:fill; border-radius:10px; background-color:#3b3745"><b><span style='color:#F1A424'> 3 |</span></b> <b>Summarization of Tech Reviews 
</b></div>


In [1]:
# Initialize the summarization pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Function to chunk text
def chunk_text(text, max_length=1024):
    words = text.split()
    chunks = []
    current_chunk = []
    current_length = 0
    
    for word in words:
        current_length += len(word) + 1  
        if current_length > max_length:
            chunks.append(" ".join(current_chunk))
            current_chunk = [word]
            current_length = len(word) + 1
        else:
            current_chunk.append(word)
    
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    
    return chunks

# Function to summarize content
def summarize_text(text):
    if not isinstance(text, str) or len(text) == 0:
        return ""
    
    chunks = chunk_text(text)
    summarized_chunks = []
    
    for chunk in chunks:
        input_length = len(chunk.split())
        max_length = min(math.ceil(input_length * 0.3), 150)
        min_length = min(max_length - 10, 30)  
        
        if max_length <= 10:  
            summarized_chunks.append(chunk)
        else:
            summary = summarizer(chunk, max_length=max_length, min_length=min_length, do_sample=False)
            summarized_chunks.append(summary[0]['summary_text'])
    
    final_summary = " ".join(summarized_chunks)
    return final_summary

# Apply the summarization function to each row in the DataFrame with progress tracking
summaries = []
for idx, row in tqdm(df.iterrows(), total=df.shape[0], desc="Summarizing content"):
    processor = row['Processor']
    source = row['Source']
    content = row['Content']
    summary = summarize_text(content)
    summaries.append({'Processor': processor, 'Source': source, 'Summary': summary})

# Create a DataFrame with the summaries
output_df = pd.DataFrame(summaries)

# Save the summarized content into a new CSV file
output_df.to_csv(output_file, index=False, encoding='utf-8')

print("Summarization complete and saved to", output_file)

Summarizing content: 100%|██████████| 44/44 [56:02<00:00, 76.43s/it] 

Summarization complete and saved to output2.csv


                                   This concludes the analysis presented in this notebook.
